# Téléportation d'un état quantique

![title](../figures/teleportation.png)

## 1. Création du circuit

**Principe des mesures différées:** On peut toujours placer les mesures en fin de circuit et les remplacer dans le circuit par des opérateurs contrôlés.

**Principe des mesures implicites:** On peut toujours supposer qu'une ligne d'un circuit quantique est terminée par une mesure, sans perte de généralité. 

Voir Nielsen et Chuang pages 186-187.

In [ ]:
from qiskit.circuit import QuantumCircuit
from math import pi

circuit = QuantumCircuit(3)

# Construction de l'état à téléporter (ici |1> )
circuit.x(0)
#circuit.u(pi/3,0,0,0)

# Construction de la paire de Bell
circuit.h(1)
circuit.cx(1,2)

# Téléportation partie 1: CNOT et Hadamard
circuit.barrier()
circuit.cx(0,1)
circuit.h(0)

# Téléportation partie 2: contrôle
circuit.cx(1,2)
circuit.cz(0,2)

circuit.measure_all()

#circuit.draw()
circuit.draw(output="mpl", filename="circuit.pdf")

## 2. Choix de la platforme d'exécution

Plusieurs choix possible:
1. Réel processeur quantique
2. Faux processeur quantique (simulateur qui partage des contraintes similaires: bruit et nombre limité de qubits)
3. Simulateur AER

In [ ]:
# from qiskit_ibm_runtime import QiskitRuntimeService
# service = QiskitRuntimeService(channel="ibm_quantum_platform", token="DL3Y9nQgo30ZGzyfv5Vo36spm7TaK2bsLITzOH9CGSff")

# backend = service.least_busy(simulator=False, operational=True)

In [ ]:
# from qiskit_ibm_runtime.fake_provider import FakeManilaV2
# backend = FakeManilaV2()

In [ ]:
from qiskit_aer import AerSimulator
backend = AerSimulator()

## 3. La "compilation"

In [ ]:
from qiskit.compiler import transpile
circuit_transpiled = transpile(circuit, backend, optimization_level=3)

print("Transpiled for backend =", backend.name)
circuit_transpiled.draw(output="mpl", filename="circuit_transpiled.pdf")

## 4. Exécution

In [ ]:
from qiskit_ibm_runtime import SamplerV2 as Sampler
sampler = Sampler(backend)
job = sampler.run([circuit_transpiled], shots=1024)

job_id = job.job_id()
print("Job id: ", job_id)

## 5. Analyse des résultats

In [ ]:
#job = service.job(job_id)
result = job.result()

from qiskit.visualization import plot_histogram
plot_histogram(result[0].data.meas.get_counts())

## 6. Exercice: détailler les étapes intermédiaires

In [ ]:
from qiskit.quantum_info import Statevector, Operator
from qiskit.circuit import QuantumCircuit
from numpy import sqrt, pi, cos, sin

theta = pi/3
u = Statevector([cos(theta/2), sin(theta/2)])
display(u.draw("latex"))

In [ ]:
from display_utils import my_display_of_tensor_products

bell = Statevector([1/sqrt(2), 0, 0, 1/sqrt(2)])
my_display_of_tensor_products([bell, u])

etat_initial = bell ^ u
display(etat_initial.draw("latex"))

In [ ]:
circuit = QuantumCircuit(3)
circuit.cx(0,1)

etat_intermediaire = etat_initial.evolve(Operator(circuit))
display(etat_intermediaire.draw("latex"))

In [ ]:
from qiskit.quantum_info import schmidt_decomposition
from display_utils import my_display_of_schmidt_decomposition

res = schmidt_decomposition(etat_intermediaire, [0,1]) 
my_display_of_schmidt_decomposition(res)

In [ ]:
circuit = QuantumCircuit(3)
circuit.cx(0,1)
circuit.h(0)

etat_intermediaire = etat_initial.evolve(Operator(circuit))
display(etat_intermediaire.draw("latex"))

res = schmidt_decomposition(etat_intermediaire, [0,1]) 
my_display_of_schmidt_decomposition(res)

In [ ]:
circuit = QuantumCircuit(3)
circuit.cx(0,1)
circuit.h(0)
circuit.cx(1,2)

etat_intermediaire = etat_initial.evolve(Operator(circuit))
display(etat_intermediaire.draw("latex"))

res = schmidt_decomposition(etat_intermediaire, [0,1]) 
my_display_of_schmidt_decomposition(res)

In [ ]:
circuit = QuantumCircuit(3)
circuit.cx(0,1)
circuit.h(0)
circuit.cx(1,2)
circuit.cz(0,2)

etat_final = etat_initial.evolve(Operator(circuit))
display(etat_final.draw("latex"))

res = schmidt_decomposition(etat_final, [0,1]) 
my_display_of_schmidt_decomposition(res)